# Session 2 Homework
For this homework you will need to first connect to the MongoDB database you have been using throughout the course.   We assume you know how to do that now.

## Question 1:
Print a projection of the entire content of the source collection in tabular form using pandas.   The table should include only the following attributes: lat, lon, depth, time, magnitude.   (Note it is wrong if the table has the "_id" field included.)   

In [1]:
# Delete me - for testing only
from mspasspy.db.database import Database   # This isn't strictly needed but used here because db set below is an instance of this class
import mspasspy.client as msc
dbclient=msc.Client()
db = dbclient.get_database('Earthscope2024')

A solution for 1

In [9]:
import pandas as pd
projection={'lat':1,'lon':1,'depth':1,'time':1,'magnitude':1,'_id':0}
cursor=db.source.find({},projection)
doclist=[]
for doc in cursor:
    doclist.append(doc)
df = pd.DataFrame.from_dict(doclist)
print(df)

        lat       lon  depth          time  magnitude
0   -7.5278  146.8140  128.5  1.323839e+09        7.1
1  -14.5572  -76.1207   29.0  1.319828e+09        7.0
2   38.7294   43.4465    7.6  1.319366e+09        7.1
3  -28.8809 -176.0332   34.8  1.319220e+09        7.4
4  -21.5930 -179.3240  629.0  1.316115e+09        7.3
5  -20.6279  169.7780  136.6  1.315091e+09        7.0
6   -7.6203  -74.5380  149.3  1.314208e+09        7.0
7  -18.3312  168.2258   31.5  1.313864e+09        7.0
8  -18.2774  168.0670   34.6  1.313859e+09        7.1
9   38.0553  143.3016   24.7  1.310259e+09        7.0
10 -29.3073 -176.2572   25.4  1.309979e+09        7.6
11  51.9805 -171.8201   49.8  1.308885e+09        7.3
12  38.2513  141.7296   53.2  1.302187e+09        7.1
13  38.0510  144.6297   19.8  1.299825e+09        7.6
14  36.2274  141.0880   25.4  1.299824e+09        7.9
15  38.2963  142.4980   19.7  1.299822e+09        9.1
16  38.4407  142.9803   26.2  1.299639e+09        7.3
17  28.6831   63.9948   79.9

## Question 2: 
Modify the output to print the origin times with obspy's UTCDateTime.  i.e. convert the times to a human readable form. Also produce the output in time order from earliest to latest.

In [11]:
from obspy import UTCDateTime
# solution
cursor=db.source.find({},projection).sort('time')
doclist=[]
for doc in cursor:
    doclist.append(doc)
    doc['time'] = UTCDateTime(doc['time'])
df = pd.DataFrame.from_dict(doclist)
print(df)

        lat       lon  depth                         time  magnitude
0  -26.8513  -63.2373  584.3  2011-01-01T09:56:58.460000Z        7.0
1  -38.3907  -73.3993   24.4  2011-01-02T20:20:18.170000Z        7.1
2   28.6831   63.9948   79.9  2011-01-18T20:23:25.570000Z        7.2
3   38.4407  142.9803   26.2  2011-03-09T02:45:19.590000Z        7.3
4   38.2963  142.4980   19.7  2011-03-11T05:46:23.200000Z        9.1
5   36.2274  141.0880   25.4  2011-03-11T06:15:37.570000Z        7.9
6   38.0510  144.6297   19.8  2011-03-11T06:25:50.740000Z        7.6
7   38.2513  141.7296   53.2  2011-04-07T14:32:44.100000Z        7.1
8   51.9805 -171.8201   49.8  2011-06-24T03:09:38.920000Z        7.3
9  -29.3073 -176.2572   25.4  2011-07-06T19:03:20.470000Z        7.6
10  38.0553  143.3016   24.7  2011-07-10T00:57:10.910000Z        7.0
11 -18.2774  168.0670   34.6  2011-08-20T16:55:04.090000Z        7.1
12 -18.3312  168.2258   31.5  2011-08-20T18:19:24.610000Z        7.0
13  -7.6203  -74.5380  149.3  2011

## Question 3
Alter your previous python code to only include sources with an origin time between Jan. 1, 2011 and the end of March (Mar. 31, 2011).   (Hint:   MsPASS stores time as a unix epoch time which can be obtained from a UTCDateTime object with the "timestamp" method.)

In [13]:
# solution
tstart = UTCDateTime('2011-01-01T00:00:00.0')
tend = UTCDateTime('2011-04-01T00:00:00.0')
query={}
query['time'] = { '$gt' : tstart.timestamp, '$lt' : tend.timestamp}
n = db.source.count_documents(query)
print(n)
cursor=db.source.find(query,projection).sort('time')
doclist=[]
for doc in cursor:
    doclist.append(doc)
    doc['time'] = UTCDateTime(doc['time'])
df = pd.DataFrame.from_dict(doclist)
print(df)

7
       lat       lon  depth                         time  magnitude
0 -26.8513  -63.2373  584.3  2011-01-01T09:56:58.460000Z        7.0
1 -38.3907  -73.3993   24.4  2011-01-02T20:20:18.170000Z        7.1
2  28.6831   63.9948   79.9  2011-01-18T20:23:25.570000Z        7.2
3  38.4407  142.9803   26.2  2011-03-09T02:45:19.590000Z        7.3
4  38.2963  142.4980   19.7  2011-03-11T05:46:23.200000Z        9.1
5  36.2274  141.0880   25.4  2011-03-11T06:15:37.570000Z        7.9
6  38.0510  144.6297   19.8  2011-03-11T06:25:50.740000Z        7.6
